# **Instrucciones Generales:**

Para cada ejercicio, realizar:

1. **Formular el problema** explícitamente en términos de los componentes requeridos por `aima-python` (`Problem class`):
  * `initial`: El estado inicial.
  * `goal`: La(s) condición(es) que definen un estado objetivo (o un método `is_goal(state)`).
  * `actions(state)`: Función que devuelve las acciones aplicables en un estado state.
  * `result(state, action)`: Función que devuelve el estado resultante de aplicar action en state.
  * `action_cost(state1, action, state2)`: Función que devuelve el costo de aplicar action para ir de `state1` a `state2`. Asumir costo 1 si no se especifica lo contrario.

2. **Elegir el algoritmo de búsqueda ciega más adecuado** (BFS, UCS, DFS, DLS, IDS) del módulo search de aima-python y **justificar tu elección** basándote en las características del problema (costos uniformes/variables, profundidad esperada de la solución, completitud, optimalidad requerida, memoria).

3. **Implementar la solución** creando una subclase de Problem y utilizando el algoritmo elegido para encontrar una solución.

4. **Presentar la solución encontrada** (la secuencia de acciones o el estado final) y **analizar brevemente el rendimiento** (nodos expandidos, costo de la ruta si aplica).

5. **Discutir las limitaciones** del enfoque de búsqueda ciega para este problema particular y si otros algoritmos (informados, etc.) podrían ser más eficientes (aunque no necesites implementarlos).

In [2]:
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# **Ejercicio 1: Descifrado de Sustitución Monoalfabética Parcial con Diccionario**

* **Escenario:** Se te da un texto cifrado corto usando un cifrado de sustitución monoalfabética (cada letra del alfabeto se mapea a otra letra única). También se te da un diccionario de palabras válidas y un fragmento del texto original (una palabra o frase corta que sabes que aparece en el texto descifrado).

* **Objetivo:** Encontrar una clave de sustitución parcial (un mapeo para un subconjunto de letras) que sea consistente y permita descifrar correctamente el fragmento conocido dentro del texto cifrado. No necesitas encontrar la clave completa ni descifrar todo el texto.

* **Estado:** El estado podría ser el mapeo parcial actual (e.g., un diccionario `{ 'A': 'X', 'B': 'Z', ... }`) y quizás las letras del alfabeto aún no asignadas.

* **Acciones:** Proponer una nueva asignación para una letra aún no mapeada en el cifrado (e.g., "asignar la letra cifrada 'Q' a la letra plana 'E'"), asegurándose de que no viole las restricciones de sustitución (una letra cifrada solo puede mapear a una letra plana, y viceversa).

* **Transición:** Añadir la nueva asignación al mapeo parcial.

* **Objetivo:** El estado es objetivo si el mapeo parcial actual, cuando se aplica al texto cifrado, revela correctamente el fragmento conocido y no genera inconsistencias obvias (e.g., mapear dos letras cifradas diferentes a la misma letra plana).

* **Complejidad:**
  * El espacio de búsqueda es el de las posibles asignaciones parciales, que crece factorialmente.
  
  * El test de objetivo implica aplicar el mapeo parcial al texto cifrado y verificar la aparición del fragmento conocido.
  
  * DFS podría perderse explorando ramas muy profundas de asignaciones incorrectas. BFS encontraría la clave parcial que requiere el menor número de asignaciones para revelar el fragmento, mientras que IDS combinaría beneficios.
  
* **Tarea:** Formula el problema. Elige entre BFS, DFS o IDS, justificando por qué uno podría ser preferible sobre los otros para encontrar una solución válida rápidamente o la que requiere menos asignaciones. Implementa y encuentra un mapeo parcial válido.

In [ ]:
from collections import deque
from collections import defaultdict

class Problem(object):
    """Clase base para problemas de búsqueda"""
    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds)
        
    def actions(self, state): raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state): return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node): return 0

class SubstitutionCipherProblem(Problem):
    def __init__(self, ciphertext, known_phrase, dictionary):
        self.cipher_words = ciphertext.upper().split()
        self.known_words = known_phrase.upper().split()
        self.dictionary = set(word.upper() for word in dictionary)
        super().__init__(initial={})

    def actions(self, state):
        """Enfoca las asignaciones en palabras que podrían contener el texto conocido"""
        for i, cipher_word in enumerate(self.cipher_words):
            for j, known_word in enumerate(self.known_words):
                if len(cipher_word) == len(known_word):
                    for cipher_char, plain_char in zip(cipher_word, known_word):
                        if cipher_char.isalpha() and cipher_char not in state:
                            return [(cipher_char, plain_char)]
        return []

    def result(self, state, action):
        """Implementación concreta del método requerido"""
        new_state = state.copy()
        cipher_char, plain_char = action
        new_state[cipher_char] = plain_char
        return new_state

    def is_goal(self, state):
        decrypted_words = [self.decrypt_word(word, state) for word in self.cipher_words]
        decrypted_text = ' '.join(decrypted_words)
        return all(word in decrypted_text for word in self.known_words)

    def decrypt_word(self, word, mapping):
        return ''.join(mapping.get(c, '_') for c in word)

def best_first_search(problem, max_depth=20):
    frontier = deque([Node(problem.initial)])
    explored = set()
    
    while frontier:
        node = frontier.popleft()
        if len(node.state) > max_depth:
            continue
            
        print(f"\nNodo actual: {node.state}")
        decrypted = ' '.join(problem.decrypt_word(word, node.state) for word in problem.cipher_words)
        print(f"Texto descifrado: {decrypted}")

        if problem.is_goal(node.state):
            print("¡Solución encontrada!")
            return node
            
        explored.add(frozenset(node.state.items()))
        
        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if frozenset(child_state.items()) not in explored:
                frontier.append(Node(child_state, node, action))
                print(f"Añadiendo asignación: {action[0]} → {action[1]}")
    
    print("BFS terminó sin solución.")
    return failure

ciphertext = "QVJAB QU LOVO"
known_phrase = "ESTOY EN"
dictionary = ["ESTOY", "EN", "CASA", "PES"]
problem = SubstitutionCipherProblem(ciphertext, known_phrase, dictionary)
solution = best_first_search(problem)

if solution != failure:
    print("\nMapeo encontrado:")
    for cipher, plain in solution.state.items():
        print(f"{cipher} → {plain}")
    
    decrypted = ' '.join(problem.decrypt_word(word, solution.state) for word in problem.cipher_words)
    print("\nTexto completo descifrado:", decrypted)


Nodo actual: {}
Texto descifrado: _____ __ ____
Añadiendo asignación: Q → E

Nodo actual: {'Q': 'E'}
Texto descifrado: E____ E_ ____
Añadiendo asignación: V → S

Nodo actual: {'Q': 'E', 'V': 'S'}
Texto descifrado: ES___ E_ __S_
Añadiendo asignación: J → T

Nodo actual: {'Q': 'E', 'V': 'S', 'J': 'T'}
Texto descifrado: EST__ E_ __S_
Añadiendo asignación: A → O

Nodo actual: {'Q': 'E', 'V': 'S', 'J': 'T', 'A': 'O'}
Texto descifrado: ESTO_ E_ __S_
Añadiendo asignación: B → Y

Nodo actual: {'Q': 'E', 'V': 'S', 'J': 'T', 'A': 'O', 'B': 'Y'}
Texto descifrado: ESTOY E_ __S_
Añadiendo asignación: U → N

Nodo actual: {'Q': 'E', 'V': 'S', 'J': 'T', 'A': 'O', 'B': 'Y', 'U': 'N'}
Texto descifrado: ESTOY EN __S_
¡Solución encontrada!

Mapeo encontrado:
Q → E
V → S
J → T
A → O
B → Y
U → N

Texto completo descifrado: ESTOY EN __S_


## Justificación del uso de BFS

### ¿Por qué BFS es mejor en este caso?

- **Encuentra la solución con el menor número de asignaciones**  
  Es una estrategia **óptima** en términos de profundidad.

- **Evita perder tiempo en ramas profundas e incorrectas**  
  A diferencia de DFS, BFS se mantiene en niveles superficiales antes de expandirse.

- **Es más eficiente que IDS en este contexto**  
  BFS **no repite iteraciones**, mientras que IDS sí explora los mismos nodos varias veces.

---

### Comparación con otros métodos

- **DFS (Búsqueda en Profundidad)**:
  - Puede encontrar soluciones **no óptimas**.
  - Tiende a **profundizar en ramas erróneas**, lo cual puede ser ineficiente.
  - Útil cuando:
    - El espacio de búsqueda es **muy grande**.
    - La solución está **muy profunda**.
  - **No ideal aquí** por la falta de optimalidad y el tamaño del espacio.

- **DLS (Búsqueda en Profundidad con Límite)**:
  - Introduce un límite de profundidad para evitar bucles infinitos.
  - Aún así, puede perder la solución si el límite no es suficiente.

- **IDS (Búsqueda en Profundidad Iterativa)**:
  - Combina DFS y BFS: garantiza optimalidad, pero **repite exploraciones**.
  - Es útil en:
    - Grafos **infinitos** o con **profundidad desconocida**.
  - En este caso, **BFS es más directo y eficiente**.

---


# **Ejercicio 2: Reconfiguración de Red de Servidores Modulares**

* **Escenario:** Tienes una red de N servidores interconectados. Hay M servicios (e.g., base de datos, web server, API gateway) que deben ejecutarse en estos servidores. Cada servidor tiene una capacidad máxima (e.g., CPU, RAM, o simplemente un número máximo de servicios que puede alojar). Cada servicio tiene unos requisitos (puede necesitar correr solo, o puede necesitar estar en el mismo servidor que otro servicio, o no puede estar en el mismo servidor que otro). Hay un estado inicial de qué servicio está en qué servidor.

* **Objetivo:** Encontrar una secuencia de movimientos de servicios entre servidores para llegar a una configuración final que satisfaga un conjunto de restricciones dadas (e.g., el Servicio A debe estar en el Servidor 1, los Servicios B y C deben estar en servidores diferentes, ningún servidor debe exceder su capacidad, todos los servicios deben estar alojados). Minimizar el número de movimientos es un objetivo secundario (o primario si se pide explícitamente).

* **Estado:** Una representación de qué servicio está asignado a qué servidor (e.g., un diccionario `{Servicio1: ServidorA, Servicio2: ServidorB, ...}`).

* **Acciones:** Mover un servicio `S` del servidor Origen al servidor `Destino`. Esta acción solo es válida si el servidor Destino tiene capacidad suficiente para alojar S (considerando los servicios que ya tiene).

* **Transición:** Actualizar la asignación del servicio S en el estado.

* **Objetivo:** El estado es objetivo si todas las restricciones de configuración y capacidad están satisfechas.

* **Costo:** Asumir costo 1 por cada movimiento (si se busca minimizar movimientos).

* **Complejidad:**
  * El estado es complejo (asignación completa). El espacio de estados es N^M en principio, aunque muchas configuraciones son inválidas.
  * Validar una acción (chequear capacidad) y verificar el estado objetivo (chequear todas las restricciones) requiere lógica adicional.
  * Si se busca el mínimo número de movimientos, BFS o IDS son apropiados. Si solo se busca una configuración válida, DFS podría ser más rápido (si tiene suerte) pero corre el riesgo de explorar caminos muy largos o infinitos si hay ciclos de movimientos.

* **Tarea:** Formula el problema, incluyendo las reglas de validación de acciones y el test de objetivo. Elige y justifica un algoritmo (probablemente BFS o IDS si quieres mínimos movimientos, o DFS si solo buscas una solución y quieres discutir sus riesgos). Implementa y encuentra una secuencia de movimientos. Analiza la complejidad y si un algoritmo ciego es práctico para instancias grandes de este problema.

In [ ]:
from collections import defaultdict, deque
import sys

class ServerReconfigProblem(Problem):
    def __init__(self, initial_assign, constraints, server_caps, services_reqs):
        self.constraints = constraints
        self.server_caps = server_caps
        self.services_reqs = services_reqs
        super().__init__(initial=initial_assign)

    def actions(self, state):
        """Genera movimientos válidos de servicios entre servidores"""
        
        valid_actions = []
        
        server_load = defaultdict(int)
        for serv, server in state.items():
            server_load[server] += self.services_reqs[serv]
        
        for service, current_server in state.items():
            for target_server in self.server_caps:
                if target_server != current_server:
                    # Verificar capacidad
                    required = self.services_reqs[service]
                    if server_load[target_server] + required <= self.server_caps[target_server]:
                        valid_actions.append((service, current_server, target_server))
        
        return valid_actions

    def result(self, state, action):
        """Aplica el movimiento"""

        service, _, target_server = action
        new_state = state.copy()
        new_state[service] = target_server
        return new_state

    def is_goal(self, state):
        """Verifica todas las restricciones"""

        server_load = defaultdict(int)
        for serv, server in state.items():
            server_load[server] += self.services_reqs[serv]
            if server_load[server] > self.server_caps[server]:
                return False
                
        for constr in self.constraints:
            if constr[0] == "co-locate":
                s1, s2 = constr[1]
                if state[s1] != state[s2]:
                    return False
            elif constr[0] == "separate":
                s1, s2 = constr[1]
                if state[s1] == state[s2]:
                    return False
            elif constr[0] == "fixed":
                s, server = constr[1], constr[2]
                if state[s] != server:
                    return False
                    
        return True

    def action_cost(self, s, a, s1):
        return 1  # Costo uniforme por movimiento
    
def breadth_first_search(problem):
    """BFS para encontrar solución con mínimo movimientos"""
    frontier = deque([Node(problem.initial)])
    explored = set()
    
    while frontier:
        node = frontier.popleft()
        if problem.is_goal(node.state):
            return node
        explored.add(frozenset(node.state.items()))
        
        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if frozenset(child_state.items()) not in explored:
                frontier.append(Node(child_state, node, action, node.path_cost + 1))
    
    return failure

def depth_limited_search(problem, limit=10):
    """Busqueda en profundidad limitada"""
    frontier = [Node(problem.initial)]
    result = failure
    
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(path_actions(node)) >= limit:
            result = cutoff
        else:
            for action in problem.actions(node.state):
                child_state = problem.result(node.state, action)
                frontier.append(Node(child_state, node, action, node.path_cost + 1))
    
    return result

def iterative_deepening_search(problem):
    """IDS combinando BFS y DFS"""
    for depth in range(1, sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != cutoff:
            return result
    return failure


# Configuración inicial
initial_assign = {
    'DB_Oracle': 'Server1',
    'DB_Replica': 'Server4',
    'Web_Frontend': 'Server2',
    'API_Gateway': 'Server1',
    'Auth_Service': 'Server3',
    'Cache_Redis': 'Server1',  
    'SOA': 'Server5',
    'OSB': 'Server6',
    'EAI': 'Server4',
    'Notification_Service': 'Server2'  
}

# Restricciones complejas
constraints = [
    ("co-locate", ['DB_Oracle', 'DB_Replica']),  
    ("separate", ['Web_Frontend', 'API_Gateway']), 
    ("fixed", 'OSB', 'Server6'),       
    ("separate", ['Auth_Service', 'Cache_Redis']), 
    ("capacity-group", ['EAI', 'SOA'])  
]

# Capacidades de servidores (en unidades de recursos)
server_caps = {
    'Server1': 8,  
    'Server2': 4,
    'Server3': 6,  
    'Server4': 8,
    'Server5': 4,
    'Server6': 10  
}

# Requisitos de recursos por servicio
services_reqs = {
    'DB_Oracle': 4,
    'DB_Replica': 3,
    'Web_Frontend': 2,
    'API_Gateway': 2,
    'Auth_Service': 3, 
    'Cache_Redis': 3,
    'SOA': 2,
    'OSB': 5, 
    'EAI': 4,
    'Notification_Service': 1
}

problem = ServerReconfigProblem(initial_assign, constraints, server_caps, services_reqs)

# Resolver con BFS 
print("Usando BFS:")
solution_bfs = breadth_first_search(problem)
if solution_bfs != failure:
    print(f"Solución encontrada en {solution_bfs.path_cost} movimientos:")
    for action in path_actions(solution_bfs):
        print(f"Mover {action[0]} de {action[1]} a {action[2]}")
else:
    print("No se encontró solución con BFS")

# Resolver con IDS
print("\nUsando IDS:")
solution_ids = iterative_deepening_search(problem)
if solution_ids != failure and solution_ids != cutoff:
    print(f"Solución encontrada en {solution_ids.path_cost} movimientos:")
    for action in path_actions(solution_ids):
        print(f"Mover {action[0]} de {action[1]} a {action[2]}")
else:
    print("No se encontró solución con IDS")

Usando BFS:
Solución encontrada en 2 movimientos:
Mover EAI de Server4 a Server6
Mover DB_Oracle de Server1 a Server4

Usando IDS:
Solución encontrada en 2 movimientos:
Mover EAI de Server4 a Server6
Mover DB_Oracle de Server1 a Server4


# Análisis de Complejidad

## Espacio de Búsqueda

- En el **peor caso**, el espacio de búsqueda es del orden:  
  **O(N^M)**  
  donde:
  - **N** = número de servidores  
  - **M** = número de servicios  
- Sin embargo, **las restricciones del problema reducen drásticamente** este espacio, ya que muchas combinaciones no son válidas.

---

## Comparación: BFS vs IDS

- **BFS (Búsqueda en Amplitud)**:
  - Encuentra la **solución óptima** (mínimo número de movimientos).
  - **Mayor consumo de memoria**, ya que guarda muchos estados en memoria.

- **IDS (Búsqueda en Profundidad Iterativa)**:
  - **Más eficiente en memoria**.
  - Puede ser **más lenta**, ya que repite muchas exploraciones.

---

## Consideraciones para Instancias Grandes

- Para valores grandes de N y M (por ejemplo, **N, M > 10**),  
  los algoritmos **ciegos** como BFS o IDS **no son prácticos**.

---

## Soluciones Alternativas

- **Usar heurísticas**:  
  Aplicar algoritmos como **A\*** con una función heurística **h(n)** bien diseñada.

- **Restringir más el espacio de acciones**:  
  Implementar restricciones más estrictas dentro del método `actions()`.

- **Considerar algoritmos de satisfacción de restricciones (CSP)**:  
  Transformar el problema en uno de **CSP** para aprovechar técnicas como:
  - Forward checking
  - Constraint propagation
